In [20]:
                        #### Algorithm 
import cv2
import numpy as np
import os
import pytesseract
import random
from PIL import Image

# Specify the path of the folder containing the images
path = r"M:\OCR\data3"
        
image_files = [f for f in os.listdir(path) if f.endswith('.jpeg') or f.endswith('.png') or f.endswith('.jpg')]

if image_files:
    random_file = random.choice(image_files)
    img = cv2.imread(os.path.join(path, random_file))
       # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

       # Threshold the image
    _, thresh = cv2.threshold(gray, 0, 250, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
       #print(thresh[100])

       # Find contours in the image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
                             #### Algorithm 1-------Handwriting Recognition
                                            ##text extraction
       
        # Filter to remove the noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

        # Apply thresholding
    thresh1 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Apply dilation and erosion
    kernel = np.ones((1, 1), np.uint8)
    dilate = cv2.dilate(thresh1, kernel, iterations=1)
    erode = cv2.erode(dilate, kernel, iterations=1)

        # Perform OCR using pytesseract
    config = '--psm 11' # PSM 11 specifies that the image contains a single block of text of uniform size
    text = pytesseract.image_to_string(erode,config=config)
    #print(f"Extracted text: {text}")
    
    with open(r'M:\OCR\Report.txt', 'w') as file:    ##specify the path where you want to save the report/output file
    # Write output to file
       file.write("Extracted Text:\n")
       file.write(text)
      

     
            #### Algorithm -------Analysis of parameters and metrics
                            ###slope of text

       #Sort the contours from left to right based on their x-coordinates
       sorted_contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

       # Calculate the y-coordinate of the bottom  of the first two bounding boxes and then avg
       x1, y1, w1, h1 = cv2.boundingRect(sorted_contours[0])
       bottom_y1 = y1 + h1

       if len(sorted_contours)>1:
        x2, y2, w2, h2 = cv2.boundingRect(sorted_contours[1])
        bottom_y2 = y2 + h2
       else:
        bottom_y2=bottom_y1
    
       avg1=(bottom_y1+bottom_y2)/2
    
       # Calculate the y-coordinate of the bottom  of the last two bounding boxes and then avg   
       x4, y4, w4, h4 = cv2.boundingRect(sorted_contours[-1])
       bottom_y4 = y4 + h4
    
       if len(sorted_contours)>1:
        x3, y3, w3, h3 = cv2.boundingRect(sorted_contours[-2])
        bottom_y3 = y3 + h3
       else:
        bottom_y3=bottom_y4
    
       avg2=(bottom_y3+bottom_y4)/2

       # Calculate the difference between the y-coordinates of the bottom left and bottom right corners
       slope = avg1 - avg2
       #print(slope)
    
       file.write("Report:\n")
    
       if slope > 20:
          file.write("1) Slope of the text indicates positivity, optimism and hope in the person's behaviour.\n")
       elif slope < -7:
          file.write("1) Slope of the text indicates feelings of sadness, disappointment and negativity in the person's behaviour.\n")
       else:
          file.write("1) Slope of the text indicates that the person is rational.\n")

    
                            #### Angle of writing

       # Apply Hough transform to detect lines
       lines = cv2.HoughLines(thresh, 1, np.pi/180, 50, min_theta=np.pi/4, max_theta=3*np.pi/4)
 
       # Calculate the dominant angle of the lines
       angles = []
       for line in lines:
          for rho,theta in line:
            angle = theta * 180/np.pi
            angles.append(angle)
        
       angles = np.asarray(angles)
       median_angle = np.median(angles)
       #print("Dominant angle:", median_angle)

       if median_angle > 93:
            file.write('2) Handwriting that leans to the left indicates a restrained and closed person, who tries to be private, to '\
           'control his emotions and finds it difficult to be free and liberated.\n')
       elif median_angle < 87:
            file.write('2) Handwriting that leans to the right indicates a very sociable person who is rational but also very '\
           'sensitive and warm.\n')
       else:
          file.write("2) Straight handwriting indicates that the person is rational.\n")
    
                                ##thickness of handwriting

        # Iterate over contours and find thickness of each contour
       total_area = 0
       width = img.shape[1]

       for contour in contours:
        total_area += cv2.contourArea(contour)

      # Calculate thickness by dividing total area by image width
       thickness = total_area / width
      #print("Overall text thickness: ", thickness)

       if thickness > 30:
        file.write('3) Thick handwriting indicates a strong person with a strong and prominent personality, '\
          'who is good with commitments and takes things seriously.\n')
       else:
        file.write('3) Thin Handwriting indicates a person who is more sensitive and passive.\n')
    
    
                          ##connected handwriting

       # Apply morphological closing to connect nearby strokes
       kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
       dilated = cv2.dilate(thresh1, kernel, iterations=2)
       closed = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel, iterations=2)

       contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
       num_cc=0
       for cnt in contours:
         area = cv2.contourArea(cnt)
         if area>100:
          num_cc=num_cc+1
        
        ###calculate number of letters

      # Count the number of alphabets in the extracted text
       num_letters = 0
       for char in text:
        if char.isalpha():
         num_letters += 1
       #print(num_letters)
    
        # Calculate connectivity ratio      
       if num_letters > 1:
         connectivity_ratio = num_cc / num_letters
       else:
         connectivity_ratio = 0

     # Print result
      # print("Connectivity ratio: ", connectivity_ratio)

       if connectivity_ratio < 0.5 and connectivity_ratio > 0 :     ##you can change it according to your dataset
          file.write('4) Connected handwriting indicates a person who has good emotional expression, creativity, open minded artistry, a '\
       'strong desire for self expression and a tendency to be more spontaneous, and less organized. \n')
       else:
          file.write('4) Writing which is not connected indicates an organized, efficient and practical person, with a strong ability '\
       'to focus on details.\n')

TypeError: 'NoneType' object is not iterable